# 3D Gaussian Splatting

**🔗 Reference:** [graphdeco-inria/gaussian-splatting (GitHub)](https://github.com/graphdeco-inria/gaussian-splatting.git)

## Steps to Convert Your Custom Video into a 3D Scene using Gaussian Splatting

Follow these steps to go from a regular video to a rendered 3D scene:

---

1. **Record a video**  
   Record a video of a subject (360° - go in a single circle around the subject) using your phone and transfer that video to your PC. Ensure that your video is in `.mp4` format. If it's not, convert it to `.mp4` using some online converter (ex: https://cloudconvert.com/).  
   **NOTE**: Make sure your video is at least of 20 seconds and covers all the angles for better 3D scene construction.

1. On your local machine clone the Gaussian Splatting Repository

   ```bash
   git clone https://github.com/graphdeco-inria/gaussian-splatting.git
   ```

1. Inside the repository folder create a new folder named `dataset`. And inside `dataset` folder create another folder `input`.

1. Put the video from Step 1 inside the `dataset` folder.

1. **Get the duration of the video**  
   Run the following command in terminal at the location of the video to extract the duration:

   ```bash
   ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 <PATH_TO_VIDEO_FILE>
   ```
   **NOTE**: You first need to install `ffmpeg` package for this command to run.

1. **Calculate FPS**  
   Use the formula to calculate the value of Required FPS (Frames per second). This fps value describes the number of frames we need to sample from the video.

   ```
   fps = <TOTAL_FRAMES_NEEDED> / <DURATION>
   ```
   **NOTE**: For the purpose of this demo we will keep the total number of frames to 100.

1. **Convert video to image frames**
   Replace the frame rate as needed:

   ```bash
   ffmpeg -i <PATH_TO_VIDEO> -vf "fps=<TOTAL_FRAMES_NEEDED>/<DURATION>" <PATH_TO_OUTPUT_FOLDER>/frame_%04d.png
   ```
   * Example: Let's say the duration of the video is 15 seconds, total frames required = 100, we are currently inside the dataset folder and the name of the video file is `input_video.mp4`, then the command in this case will be: `ffmpeg -i input_video.mp4 -vf "fps=100/15" input/frame_%04d.png`. (Note that the output of this command will serve as input to the colmap hence we are naming the directory as input and referring to it as `<PATH_TO_OUTPUT_FOLDER>` in the above command.

   * After this you should get around 100 frames (need not be exactly 100) in your `input` folder.

1. **Install COLMAP** (if not already installed)  
    Refer to: https://colmap.github.io/install.html  
    **Note**: The `convert.py` script of gaussian-splatting repo requires Colmap to run.

1. **Convert images for training**
   Run the following script (from inside the repo folder: gaussian-splatting/):

   ```bash
   python3 convert.py -s dataset
   ```
   * It will take around **5-6 minutes to run** based on the number of frames.

   * This script will run COLMAP and output camera aligned images (NeRF/GS compatible format) along with camera parameters.

   * After executing the script the structure of `dataset` folder should look like this:
   ```text
   .
├── distorted
│   ├── database.db
│   └── sparse
│       └── 0
│           ├── cameras.bin
│           ├── images.bin
│           ├── points3D.bin
│           └── project.ini
├── images
│   ├── frame_0001.png
│   ├── frame_0002.png
│   ├── .
│   ├── .
│   ├── .
│   └── frame_0100.png
├── input
│   ├── distorted
│   │   └── sparse
│   ├── frame_0001.png
│   ├── frame_0002.png
│   ├── .
│   ├── .
│   ├── .
│   └── frame_0100.png
├── run-colmap-geometric.sh
├── run-colmap-photometric.sh
├── sparse
│   └── 0
│       ├── cameras.bin
│       ├── images.bin
│       └── points3D.bin
└── stereo
    ├── consistency_graphs
    ├── depth_maps
    ├── fusion.cfg
    ├── normal_maps
    └── patch-match.cfg
   ```

1. **Upload data to Google Drive**  
   * From the `dataset` folder, upload the `images/` and `sparse/` directories to Google Drive. Make sure to keep both of them at the same location.
   * The folders can be uploaded as a `.zip` for faster uploading, and then unzip later in the notebook before using the dataset.

1. Run the cells below.

---



## Mount the drive to access the data uploaded there

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Unzipping
Run this cell only if you have uploaded the folders as zip and want to unzip them otherwise skip this cell.

In [ ]:
# !unzip "/content/drive/MyDrive/3DVSS_25/3DGS/images.zip" -d "/content/drive/MyDrive/3DVSS_25/3DGS"
# !unzip "/content/drive/MyDrive/3DVSS_25/3DGS/sparse.zip" -d "/content/drive/MyDrive/3DVSS_25/3DGS"

## Cloning the Gaussian Splatting GitHub repository

In [ ]:
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

---

12. **Open the side panel** in your code editor to view the project directory structure.
13. **Navigate to** the following path:

    ```
    gaussian-splatting/submodules/simple-knn
    ```
14. **Double-click** on the file named `simple_knn.cu`. It should open in the text editor on the right panel.
15. **Add the following line** at the top of the file along with the other `#include` statements:

    ```cpp
    #include <float.h>
    ```
press `ctrl + s` or `cmd + s` to save the file and then close it.
---

## Build the wheels for submodule dependencies

> In Python, a **wheel** is a built-package format with the `.whl` extension. It's essentially a pre-built **binary distribution** of a Python package that makes installation faster and easier compared to building from source. Wheels eliminate the need to compile code during installation, which is especially useful for packages with complex dependencies or native C/C++ extensions.

In [ ]:
%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

## Select Parameters

In [ ]:
# @markdown Give absolute path to the location in your drive where the `images` and `sparse` folders are saved.
data_path="/content/drive/MyDrive/3DVSS_25/3DGS" # @param {type: 'string'}

# @markdown More the number of iterations better will be the constructed 3D scene.
iterations = 7000 # @param {type: 'integer'}

# @markdown **Note**: Training for 7000 iterations will take around 15 mins

In [ ]:
!python train.py -s "$data_path" --iterations "$iterations"

## Visualisation

---
16. **After training completes**, navigate to the following directory:

```
/content/gaussian-splatting/output/random_name/point_cloud/iteration_<n_iterations>
```
Download the `point_cloud.ply` file from there.  
**Note**: The randomly generated name of the output folder can be found in the information printed above during training.


**Visualise the output**: To visualize the generated `.ply` file follow these steps:

17. Visit: [https://niujinshuchong.github.io/mip-splatting-demo/](https://niujinshuchong.github.io/mip-splatting-demo/)

18. Scroll down to the **file upload section**.

19. Upload your `.ply` file generated from training.

20. Set the following visualization parameters:

    ```text
    Minimum alpha   : 1  
    Camera up       : 0, -1, 0  
    Camera position : 0, 1, 0  
    Camera look-at  : 1, 0, 0
    ```

21. Click on **`View`** to render and explore your 3D scene! 😄
---